In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_transformers4rec_tutorial-01-preprocess/nvidia_logo.png" style="width: 90px; float: right;">

# Preliminary Preprocessing

**Read and Process E-Commerce data**

In this notebook, we are going to use a subset of a publicly available [eCommerce dataset](https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store). The full dataset contains 7 months data (from October 2019 to April 2020) from a large multi-category online store. Each row in the file represents an event. All events are related to products and users. Each event is like many-to-many relation between products and users.
Data collected by Open CDP project and the source of the dataset is [REES46 Marketing Platform](https://rees46.com/).

We use only `2019-Oct.csv` file for training our models, so you can visit this site and download the csv file: https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store.

### Import the required libraries

In [2]:
import os
import numpy as np 
import gc
import shutil
import glob

import cudf
import nvtabular as nvt

/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")


### Read Data via cuDF from CSV

At this point we expect that you have already downloaded the `2019-Oct.csv` dataset and stored it in the `INPUT_DATA_DIR` as defined below. It is worth mentioning that the raw dataset is ~ 6 GB, therefore a single GPU with 16 GB or less memory might run out of memory.

In [3]:
# define some information about where to get our data
INPUT_DATA_DIR_SHORT = os.environ.get("INPUT_DATA_DIR_SHORT", "/workspace/data/")

In [4]:
%%time
raw_df = cudf.read_csv(os.path.join(INPUT_DATA_DIR_SHORT, '2019-Oct.csv')) 
raw_df.head()

CPU times: user 1.15 s, sys: 444 ms, total: 1.59 s
Wall time: 1.59 s


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,<NA>,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,<NA>,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [5]:
raw_df.shape

(42291197, 9)

### Convert timestamp from datetime

In [6]:
raw_df['event_time_dt'] = raw_df['event_time'].astype('datetime64[s]')
raw_df['event_time_ts']= raw_df['event_time_dt'].astype('int')
raw_df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_time_dt,event_time_ts
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,<NA>,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01 00:00:00,1569888000
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,2019-10-01 00:00:00,1569888000
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,<NA>,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,2019-10-01 00:00:01,1569888001
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,2019-10-01 00:00:01,1569888001
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,2019-10-01 00:00:04,1569888004


In [7]:
# check out the columns with nulls
raw_df.isnull().any()

event_time       False
event_type       False
product_id       False
category_id      False
category_code     True
brand             True
price            False
user_id           True
user_session      True
event_time_dt    False
event_time_ts    False
dtype: bool

In [8]:
# Remove rows where `user_session` is null.
raw_df = raw_df[raw_df['user_session'].isnull()==False]
len(raw_df)

42291194

We no longer need `event_time` column.

In [9]:
raw_df = raw_df.drop(['event_time'],  axis=1)

### Categorify `user_session` column
Although `user_session` is not used as an input feature for the model, it is useful to convert those raw long string to int values to avoid potential failures when grouping interactions by `user_session` in the next notebook.

In [10]:
cols = list(raw_df.columns)
cols.remove('user_session')
cols

['event_type',
 'product_id',
 'category_id',
 'category_code',
 'brand',
 'price',
 'user_id',
 'event_time_dt',
 'event_time_ts']

In [11]:
# load data 
df_event = nvt.Dataset(raw_df[4229119:4229119+1000]) 

# categorify user_session 
cat_feats = ['user_session'] >> nvt.ops.Categorify()

workflow = nvt.Workflow(cols + cat_feats)
workflow.fit(df_event)
df = workflow.transform(df_event).to_ddf().compute()

In [12]:
df.head()

,event_type,product_id,category_id,category_code,brand,price,user_id,event_time_dt,event_time_ts,user_session
0,view,1004853,2053013555631882655,electronics.smartphone,meizu,231.41,534420892,2019-10-04 11:25:43,1570188343,422
1,view,29000025,2053013562871251777,<NA>,gezatone,43.50,549035690,2019-10-04 11:25:43,1570188343,16
2,view,12703734,2053013553559896355,<NA>,gislaved,109.14,539488336,2019-10-04 11:25:43,1570188343,329
3,view,2701657,2053013563911439225,appliances.kitchen.refrigerators,beko,244.51,518917290,2019-10-04 11:25:43,1570188343,29
4,view,1004237,2053013555631882655,electronics.smartphone,apple,1088.45,543077219,2019-10-04 11:25:43,1570188343,171


In [13]:
raw_df = None
del(raw_df)

In [14]:
gc.collect()

635

### Removing consecutive repeated (user, item) interactions

We keep repeated interactions on the same items, removing only consecutive interactions, because it might be due to browser tab refreshes or different interaction types (e.g. click, add-to-card, purchase)

In [15]:
%%time
df = df.sort_values(['user_session', 'event_time_ts']).reset_index(drop=True)

print("Count with in-session repeated interactions: {}".format(len(df)))
# Sorts the dataframe by session and timestamp, to remove consecutive repetitions
df['product_id_past'] = df['product_id'].shift(1).fillna(0)
df['session_id_past'] = df['user_session'].shift(1).fillna(0)
#Keeping only no consecutive repeated in session interactions
df = df[~((df['user_session'] == df['session_id_past']) & \
             (df['product_id'] == df['product_id_past']))]
print("Count after removed in-session repeated interactions: {}".format(len(df)))
del(df['product_id_past'])
del(df['session_id_past'])

gc.collect()

Count with in-session repeated interactions: 1000
Count after removed in-session repeated interactions: 885
CPU times: user 65.7 ms, sys: 918 µs, total: 66.6 ms
Wall time: 65.5 ms


17

### Include the item first time seen feature (for recency calculation)

We create `prod_first_event_time_ts` column which indicates the timestamp that an item was seen first time.

In [16]:
item_first_interaction_df = df.groupby('product_id').agg({'event_time_ts': 'min'}) \
            .reset_index().rename(columns={'event_time_ts': 'prod_first_event_time_ts'})
item_first_interaction_df.head()
gc.collect()

0

In [17]:
df = df.merge(item_first_interaction_df, on=['product_id'], how='left').reset_index(drop=True)

In [18]:
df.head()

,event_type,product_id,category_id,category_code,brand,price,user_id,event_time_dt,event_time_ts,user_session,prod_first_event_time_ts
0,view,1002528,2053013555631882655,electronics.smartphone,apple,591.78,529034709,2019-10-04 11:25:56,1570188356,445,1570188356
1,view,1004837,2053013555631882655,electronics.smartphone,xiaomi,323.03,518750772,2019-10-04 11:26:15,1570188375,446,1570188375
2,view,1005157,2053013555631882655,electronics.smartphone,xiaomi,282.89,514251675,2019-10-04 11:26:04,1570188364,447,1570188364
3,view,1004750,2053013555631882655,electronics.smartphone,samsung,197.95,555442235,2019-10-04 11:26:00,1570188360,448,1570188354
4,view,1004886,2053013555631882655,electronics.smartphone,oppo,154.16,513314561,2019-10-04 11:25:50,1570188350,449,1570188350


In [19]:
del(item_first_interaction_df)
item_first_interaction_df=None
gc.collect()

18

In this tutorial, we only use one week of data from Oct 2019 dataset.

In [20]:
# check the min date
df['event_time_dt'].min()

numpy.datetime64('2019-10-04T11:25:43')

In [21]:
# Filters only the first week of the data.
df = df[df['event_time_dt'] < np.datetime64('2019-10-08')].reset_index(drop=True)

We verify that we only have the first week of Oct-2019 dataset.

In [22]:
df['event_time_dt'].max()

numpy.datetime64('2019-10-04T11:26:26')

We drop `event_time_dt` column as it will not be used anymore.

In [23]:
df = df.drop(['event_time_dt'],  axis=1)

In [24]:
df.head()

,event_type,product_id,category_id,category_code,brand,price,user_id,event_time_ts,user_session,prod_first_event_time_ts
0,view,1002528,2053013555631882655,electronics.smartphone,apple,591.78,529034709,1570188356,445,1570188356
1,view,1004837,2053013555631882655,electronics.smartphone,xiaomi,323.03,518750772,1570188375,446,1570188375
2,view,1005157,2053013555631882655,electronics.smartphone,xiaomi,282.89,514251675,1570188364,447,1570188364
3,view,1004750,2053013555631882655,electronics.smartphone,samsung,197.95,555442235,1570188360,448,1570188354
4,view,1004886,2053013555631882655,electronics.smartphone,oppo,154.16,513314561,1570188350,449,1570188350


Save the data as a single parquet file to be used in the ETL notebook.

In [25]:
# save df as parquet files on disk
df.to_parquet(os.path.join(INPUT_DATA_DIR_SHORT, 'Oct-2019_SHORT.parquet'))

- Shut down the kernel

In [26]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}